In [3]:
 import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
from pathlib import Path
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from lockdowndates.core import LockdownDates

from catboost import CatBoostRegressor
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
from datetime import timedelta, date

In [28]:
X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam

X_train, y_train, X_test, y_test = utils.train_test_split_temporal(X, y, delta_threshold="30 days")


num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id',]

col_transformer = ColumnTransformer(
    transformers=[
        
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(sparse=False), cat_features)
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(
    FunctionTransformer(utils.prepro),
    col_transformer
)

X_train_scaled = pipeline.fit_transform(X_train)

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train_scaled.shape[0], X_train_scaled.shape[1])))
model.add(LSTM(50))
model.add(Dense(1))

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train_scaled, y_train, epochs=15, batch_size=32, validation_data=(pipeline.transform(X_test),y_test))

Epoch 1/15


ValueError: in user code:

    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/utils/traceback_util

In [46]:
X_test_scaled = pipeline.transform(X_test)  # Assurez-vous que X_test est votre ensemble de données de test

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test_scaled)

1260/1260 [==============================] - 0s 267us/step


In [47]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 1.4865370047426993


In [26]:
y_pred

array([[4.2675867],
       [2.0292158],
       [4.328022 ],
       ...,
       [4.15152  ],
       [2.1412816],
       [1.2740922]], dtype=float32)

In [57]:
X = utils.prepro(X)

In [61]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Définition des caractéristiques numériques et catégorielles
num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id']

# Création du ColumnTransformer
col_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(sparse=False), cat_features)
    ],
    remainder='passthrough'
)

# Application du ColumnTransformer sur les données X
X_transformed = col_transformer.fit_transform(X)

# Convertir X_transformed en un format compatible avec RNN si nécessaire
# X_transformed = convert_to_sequences(X_transformed)  # À définir en fonction de vos données

# Séparation des données en ensembles d'entraînement et de test
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Création du modèle RNN
model_rnn = Sequential()
model_rnn.add(LSTM(50, return_sequences=True, input_shape=(X_train_rnn.shape[0], X_train_rnn.shape[1])))
model_rnn.add(LSTM(50))
model_rnn.add(Dense(1))

# Compilation du modèle
model_rnn.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model_rnn.fit(X_train_rnn, y_train_rnn, epochs=100, batch_size=32, validation_data=(X_test_rnn, y_test_rnn))

Epoch 1/100


ValueError: in user code:

    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/raphaelamzallag/.pyenv/versions/3.10.6/envs/final_pfds/lib/python3.10/site-packages/keras/src/utils/traceback_util